## Path Association ##

In [7]:
import cv2
import os
import shutil
import pandas as pd
import json
import numpy as np
import cv2
import math
import random
import sys

sys.path.append(r'../PeterPan/')

import YoloTools as yt

bb_test = [ [4,1],[7,6] ,1 ]
bb_gt = [ [1,4],[5,8] ,1 ]
print(yt.iou(bb_test,bb_gt))

0.5559289377722367


In [8]:
# import numpy as np
# import matplotlib.pyplot as plt
# import DasPrep as dp
# import glob, os
# import datetime
# import h5py
# from scipy.signal import decimate

# mutation_stamp = [6, 182, 700, 722, 764, 788, 850, 1026, 1084, 1170, 1268, 1290, 1410, 1446, 1460, 1630, 1650, 1696, 1780, 1852, 1938, 2038, 2234,
#                     2392, 2540, 2606, 2682, 2700, 2742, 3050, 3496, 3528, 3640, 3716, 3856, 3968]
# fiber_cum_len = [i*4 for i in mutation_stamp]
# dt = 1./500
# dx = 4
# M = np.array([10, 5])
# Mdt = dt*np.prod(M)


# npylist = glob.glob('../../Highway_2024_0510/output/0628_data2/deci_7_7/npy/*.npy')
# print(len(npylist))
# npylist.sort()

# path = '../../Highway_2024_0510/output/0628_data2/deci_7_7/pic4/'
# if not os.path.exists(path):
#     os.makedirs(path)

# num = 1
# # for k in range(0,len(npylist)//num - 1):
# for k in range(0, 10):

#     data = np.concatenate([np.load(npylist[i]) for i in range(k*num,(k+1)*num)], axis=-1)
#     data = dp.das_preprocess(data)
#     dataEW = data[int(fiber_cum_len[21]/dx):int(fiber_cum_len[22]/dx), :]
#     dataEW_d = np.delete(dataEW, np.r_[np.arange(51,60), np.arange(81,94), np.arange(160,196)], axis=0)
#     # dataEW_d = np.delete(dataEW, np.r_[np.arange(51,60), np.arange(81,94), np.arange(160,165)], axis=0)

#     data2 = dp.tapering(dataEW_d, 0.2)
#     data2 = dp.lowpass(data2, Mdt,  fh=2)
#     data2 = dp.das_preprocess(data2)
#     data2 = decimate(data2, 2).astype('float')
#     data2 = data2 / (data2).std(axis=1, keepdims=True)
#     plt.figure(figsize=[5.12,6.4])
#     # plt.figure(figsize=[6.4,6.4]) 
#     # plt.figure(figsize=[16,16]) 

#     data_plot = data2[:, :]
#     clim = data_plot.std() * 1
#     plt.imshow(data_plot.T, aspect='auto', cmap='seismic', vmin=-clim, vmax=clim)
#     plt.savefig(path+str(k)+'.png' )
#     plt.close()

# print(data2.shape)
# # 5Hz 约700m 30s*3

In [9]:
sys.path.append(r'../PeterPan/')
import eval_yolo_results2
import YoloTools as yt
import glob, os

# --------------可视化线段关联验证流程, 5个步骤依次运行-------------------
img_path = r'../../Highway_2024_0510/output/0628_data2/deci_7_7/pic/0.png' 
patch_dir =    r'../../Highway_2024_0510/output/0628_data2/deci_7_7/patches4'
yolo_predict = r'../../Highway_2024_0510/output/0628_data2/deci_7_7/yolo/labels'   # 每次预测完会存入新的文件夹，需修改“exp”对应的数字

img_flist = glob.glob('../../Highway_2024_0510/output/0628_data2/deci_7_7/pic4/*.png')
# 1.img to patch, 待验证图像划分patches保存
if not os.path.exists(patch_dir):
    os.makedirs(patch_dir)
for img_path in img_flist:
    pathc_num, img_w,img_h= eval_yolo_results2.img2patch_pad(img_path, patch_dir, patch_size=64, padding=16)


In [10]:
# 2. yolo predict, 用训练好的yolo模型预测上一步骤中的patches
# 命令：python detect.py --weights ./runs/train/exp7/weights/best.pt --source ../datasets/traffic/images/val --imgsz 192 --iou-thres 0.5 --conf-thres 0.5 --save-txt
# 命令：python ../../yolo5/detect.py --weights ./runs/train/exp7/weights/best.pt --source ../../Highway_2024_0510/output/0628_data2/deci_7_7/patches --project '../../Highway_2024_0510/output/0628_data2/' --name 'deci_7_7/yolo/' --imgsz 192 --iou-thres 0.5 --conf-thres 0.5 --save-txt
# ! python ../../Vehicle_Trace_DL/yolov5/detect.py --weights ../../Vehicle_Trace_DL/yolov5/runs/train/exp15/weights/best.pt --source ../../Highway_2024_0510/output/0628_data2/deci_7_7/patches4 --project '../../Highway_2024_0510/output/0628_data2/' --name 'deci_7_7/yolo/' --imgsz 192 --iou-thres 0.5 --conf-thres 0.5 --save-txt

In [11]:
yolo_predict = r'../../Highway_2024_0510/output/0628_data2/deci_7_7/yolo13/labels'   # 每次预测完会存入新的文件夹，需修改“exp”对应的数字
img_save_dir = r"../../Highway_2024_0510/output/0628_data2/deci_7_7/predict_vis_result20"
# 3. draw predicted lines to patch, 将预测结果的txt文件合并到json文件中，json文件存放在结果文件夹中
if not os.path.exists(img_save_dir):
    os.makedirs(img_save_dir)
img_flist.sort()
print(len(img_flist))
for img_path in img_flist:
    img_name = img_path.split("/")[-1].replace(".png","")
    txt_save_path = os.path.join(img_save_dir,img_name+'.txt')
    os.remove(txt_save_path) #清空txt文件
    yt.txt2json(img_name,yolo_predict,txt_save_path,img_w,img_h ,padding=16 , patch_num = 75 , patch_size=64 ,iou_threshold = 0.2)

# txt2line(yolo_predict,patch_dir)
# 4.将patch图片进行拼接
for img_path in img_flist:
    img_name = img_path.split("/")[-1].replace(".png","")
    img_save_path = os.path.join(img_save_dir,img_name+'.png')
    yt.patch2img_pad(patch_dir,img_name, img_save_path,img_w,img_h, padding=16 , patch_num=pathc_num)

# # 5. json文件中的线段，画在拼接好的完整图片上
yt.draw_line(img_save_dir)

#395 

20
from   # 306   to  # 161
from   # 272   to  # 145
from   # 306   to  # 173
from   # 303   to  # 147
from   # 318   to  # 169
from   # 336   to  # 164
from   # 310   to  # 151
from   # 261   to  # 124
from   # 306   to  # 146
from   # 290   to  # 135
from   # 294   to  # 147
from   # 296   to  # 153
from   # 305   to  # 158
from   # 296   to  # 162
from   # 294   to  # 147
from   # 293   to  # 140
from   # 280   to  # 146
from   # 271   to  # 119
from   # 275   to  # 142
from   # 321   to  # 161
[[[122, 98], [126, 102], 1]]
[[[126, 80], [68, 126], 0]]
[[[128, 107], [105, 126], 0], [[103, 127], [97, 135], 0]]
[[[67, 110], [85, 126], 1], [[84, 127], [128, 171], 1], [[129, 170], [148, 191], 1], [[148, 193], [192, 236], 1], [[191, 235], [211, 254], 1], [[212, 255], [256, 299], 1], [[256, 301], [274, 319], 1], [[274, 321], [319, 364], 1], [[320, 367], [338, 384], 1], [[339, 386], [382, 430], 1]]
(173, 187, 176)
[[[180, 98], [148, 126], 0], [[149, 129], [128, 148], 0]]
[[[168, 79], [190, 1